<a href="https://colab.research.google.com/github/elizabethavargas/baseball/blob/main/create_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bat-Level Baseball Data

### Download Data

In [ ]:
!pip install pybaseball
from pybaseball import statcast
import pandas as pd

start_date = "2024-04-01"
end_date = "2024-05-01"
data = statcast(start_dt=start_date, end_dt=end_date)

This is a large query, it may take a moment to complete


  0%|          | 0/31 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
/usr/local/lib/python3.12/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
/usr/local/lib/python3.12/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = d

In [ ]:
# Keep relevant columns  ### data[['pitch_type', 'release_speed', 'description', 'zone', 'game_type', 'stand', 'p_throws', 'type', 'balls', 'strikes', 'pitch_number', 'age_bat',]]
df = data[['batter', 'stand', 'player_name', 'launch_speed', 'launch_angle', 'release_speed', 'age_bat', 'game_date','events', 'home_team', 'pitch_type', 'game_pk']].copy()

# Encode home_run as binary target
df['home_run'] = df['events'] == 'home_run'
df.dropna(subset=['launch_speed','launch_angle'], inplace=True)

df

,batter,stand,player_name,launch_speed,launch_angle,release_speed,age_bat,game_date,events,home_team,pitch_type,game_pk,home_run
1743,664983,L,"Varland, Gus",84.6,-23,95.0,27,2024-05-01,field_out,AZ,FF,747207,False
1826,664983,L,"Varland, Gus",72.0,52,94.4,27,2024-05-01,NaN,AZ,FF,747207,False
1914,571466,L,"Varland, Gus",73.3,-10,94.7,33,2024-05-01,field_out,AZ,FF,747207,False
2197,677942,R,"Varland, Gus",73.5,-10,89.2,25,2024-05-01,NaN,AZ,SL,747207,False
2465,682998,L,"Varland, Gus",98.5,12,89.0,24,2024-05-01,single,AZ,SL,747207,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3970,682928,L,"Gonzales, Marco",97.6,-15,90.9,24,2024-04-01,field_out,WSH,SI,744875,False
2604,457705,R,"Gore, MacKenzie",76.7,58,97.7,38,2024-04-01,NaN,WSH,FF,744875,False
3266,668804,R,"Gore, MacKenzie",90.5,14,96.9,29,2024-04-01,single,WSH,FF,744875,False
3488,656582,R,"Gore, MacKenzie",80.9,67,97.2,32,2024-04-01,field_out,WSH,FF,744875,False


### Add Altitide and Weather

In [ ]:
# add altitude
altitude_map = {
    'ARI': 1100,  # Chase Field, Phoenix, AZ
    'AZ': 1100,   # alt name for ARI
    'ATL': 1050,  # Truist Park, Atlanta, GA
    'BAL': 10,    # Oriole Park at Camden Yards, Baltimore, MD
    'BOS': 30,    # Fenway Park, Boston, MA
    'CHC': 600,   # Wrigley Field, Chicago, IL
    'CIN': 500,   # Great American Ball Park, Cincinnati, OH
    'CLE': 650,   # Progressive Field, Cleveland, OH
    'COL': 5200,  # Coors Field, Denver, CO
    'CWS': 600,   # Guaranteed Rate Field, Chicago, IL
    'DET': 600,   # Comerica Park, Detroit, MI
    'HOU': 50,    # Minute Maid Park, Houston, TX
    'KC': 1000,   # Kauffman Stadium, Kansas City, MO
    'LAA': 150,   # Angel Stadium, Anaheim, CA
    'LAD': 300,   # Dodger Stadium, Los Angeles, CA
    'MIA': 10,    # loanDepot park, Miami, FL
    'MIL': 700,   # American Family Field, Milwaukee, WI
    'MIN': 800,   # Target Field, Minneapolis, MN
    'NYM': 10,    # Citi Field, Queens, NY
    'NYY': 10,    # Yankee Stadium, Bronx, NY
    'OAK': 50,    # Oakland Coliseum, Oakland, CA
    'PHI': 10,    # Citizens Bank Park, Philadelphia, PA
    'PIT': 750,   # PNC Park, Pittsburgh, PA
    'SD': 50,     # Petco Park, San Diego, CA
    'SEA': 30,    # T-Mobile Park, Seattle, WA
    'SF': 50,     # Oracle Park, San Francisco, CA
    'STL': 450,   # Busch Stadium, St. Louis, MO
    'TB': 50,     # Tropicana Field, St. Petersburg, FL
    'TEX': 600,   # Globe Life Field, Arlington, TX
    'TOR': 500,   # Rogers Centre, Toronto, ON (approximate)
    'WSH': 10,    # Nationals Park, Washington, D.C.
}

df['altitude'] = df['home_team'].map(altitude_map)
df

,batter,stand,player_name,launch_speed,launch_angle,release_speed,age_bat,game_date,events,home_team,pitch_type,game_pk,home_run,altitude
1743,664983,L,"Varland, Gus",84.6,-23,95.0,27,2024-05-01,field_out,AZ,FF,747207,False,1100
1826,664983,L,"Varland, Gus",72.0,52,94.4,27,2024-05-01,NaN,AZ,FF,747207,False,1100
1914,571466,L,"Varland, Gus",73.3,-10,94.7,33,2024-05-01,field_out,AZ,FF,747207,False,1100
2197,677942,R,"Varland, Gus",73.5,-10,89.2,25,2024-05-01,NaN,AZ,SL,747207,False,1100
2465,682998,L,"Varland, Gus",98.5,12,89.0,24,2024-05-01,single,AZ,SL,747207,False,1100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3970,682928,L,"Gonzales, Marco",97.6,-15,90.9,24,2024-04-01,field_out,WSH,SI,744875,False,10
2604,457705,R,"Gore, MacKenzie",76.7,58,97.7,38,2024-04-01,NaN,WSH,FF,744875,False,10
3266,668804,R,"Gore, MacKenzie",90.5,14,96.9,29,2024-04-01,single,WSH,FF,744875,False,10
3488,656582,R,"Gore, MacKenzie",80.9,67,97.2,32,2024-04-01,field_out,WSH,FF,744875,False,10


In [ ]:
import requests
import pandas as pd

def get_weather_for_game(game_pk):
    """
    Fetch weather info for a given MLB game_pk from MLB StatsAPI.
    """
    url = f"https://statsapi.mlb.com/api/v1.1/game/{game_pk}/feed/live"
    resp = requests.get(url)
    if resp.status_code != 200:
        return None

    data = resp.json()
    weather_info = data.get("gameData", {}).get("weather", {})

    return {
        "temperature": weather_info.get("temp"),
        "condition": weather_info.get("condition"),
        "wind": weather_info.get("wind"),
    }

# Fetch weather for each game
game_ids = df["game_pk"].unique()
weather_data = {gid: get_weather_for_game(gid) for gid in game_ids}

# turn into dataframe
weather_df = pd.DataFrame.from_dict(weather_data, orient="index")
weather_df.index.name = "game_pk"
weather_df.reset_index(inplace=True)

# merge back into Statcast df
df = df.merge(weather_df, on="game_pk", how="left")
df


,batter,stand,player_name,launch_speed,launch_angle,release_speed,age_bat,game_date,events,home_team,pitch_type,game_pk,home_run,altitude,temperature,condition,wind
0,664983,L,"Varland, Gus",84.6,-23,95.0,27,2024-05-01,field_out,AZ,FF,747207,False,1100,88,Partly Cloudy,"9 mph, L To R"
1,664983,L,"Varland, Gus",72.0,52,94.4,27,2024-05-01,NaN,AZ,FF,747207,False,1100,88,Partly Cloudy,"9 mph, L To R"
2,571466,L,"Varland, Gus",73.3,-10,94.7,33,2024-05-01,field_out,AZ,FF,747207,False,1100,88,Partly Cloudy,"9 mph, L To R"
3,677942,R,"Varland, Gus",73.5,-10,89.2,25,2024-05-01,NaN,AZ,SL,747207,False,1100,88,Partly Cloudy,"9 mph, L To R"
4,682998,L,"Varland, Gus",98.5,12,89.0,24,2024-05-01,single,AZ,SL,747207,False,1100,88,Partly Cloudy,"9 mph, L To R"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39575,682928,L,"Gonzales, Marco",97.6,-15,90.9,24,2024-04-01,field_out,WSH,SI,744875,False,10,53,Cloudy,"4 mph, In From RF"
39576,457705,R,"Gore, MacKenzie",76.7,58,97.7,38,2024-04-01,NaN,WSH,FF,744875,False,10,53,Cloudy,"4 mph, In From RF"
39577,668804,R,"Gore, MacKenzie",90.5,14,96.9,29,2024-04-01,single,WSH,FF,744875,False,10,53,Cloudy,"4 mph, In From RF"
39578,656582,R,"Gore, MacKenzie",80.9,67,97.2,32,2024-04-01,field_out,WSH,FF,744875,False,10,53,Cloudy,"4 mph, In From RF"


In [ ]:
# encode weather conditions
encoding_map = {
    "Dome": 0,
    "Roof Closed": 0,
    "Clear": 1,
    "Sunny": 2,
    "Partly Cloudy": 3,
    "Cloudy": 4,
    "Overcast": 5,
    "Drizzle": 6
}
df["condition"] = df["condition"].map(encoding_map)

# convert wind str to int
df['wind_int'] = df.wind.str.split(' ').str[0]

### Add Barrel Column

In [ ]:
def is_barrel(row):
    ev = row['launch_speed']
    la = row['launch_angle']

    if ev < 98:
        return False

    if ev >= 116:
        return 8 <= la <= 50  # capped range
    else:
        # expansion formula
        expansion = ev - 98
        min_angle = 26 - expansion
        max_angle = 30 + expansion
        return min_angle <= la <= max_angle

df['barrel'] = df.apply(is_barrel, axis=1)
df


,batter,stand,player_name,launch_speed,launch_angle,release_speed,age_bat,game_date,events,home_team,pitch_type,game_pk,home_run,altitude,temperature,condition,wind,wind_int,barrel
0,664983,L,"Varland, Gus",84.6,-23,95.0,27,2024-05-01,field_out,AZ,FF,747207,False,1100,88,3.0,"9 mph, L To R",9,False
1,664983,L,"Varland, Gus",72.0,52,94.4,27,2024-05-01,NaN,AZ,FF,747207,False,1100,88,3.0,"9 mph, L To R",9,False
2,571466,L,"Varland, Gus",73.3,-10,94.7,33,2024-05-01,field_out,AZ,FF,747207,False,1100,88,3.0,"9 mph, L To R",9,False
3,677942,R,"Varland, Gus",73.5,-10,89.2,25,2024-05-01,NaN,AZ,SL,747207,False,1100,88,3.0,"9 mph, L To R",9,False
4,682998,L,"Varland, Gus",98.5,12,89.0,24,2024-05-01,single,AZ,SL,747207,False,1100,88,3.0,"9 mph, L To R",9,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39575,682928,L,"Gonzales, Marco",97.6,-15,90.9,24,2024-04-01,field_out,WSH,SI,744875,False,10,53,4.0,"4 mph, In From RF",4,False
39576,457705,R,"Gore, MacKenzie",76.7,58,97.7,38,2024-04-01,NaN,WSH,FF,744875,False,10,53,4.0,"4 mph, In From RF",4,False
39577,668804,R,"Gore, MacKenzie",90.5,14,96.9,29,2024-04-01,single,WSH,FF,744875,False,10,53,4.0,"4 mph, In From RF",4,False
39578,656582,R,"Gore, MacKenzie",80.9,67,97.2,32,2024-04-01,field_out,WSH,FF,744875,False,10,53,4.0,"4 mph, In From RF",4,False


In [ ]:
# encode stand for model
df.stand = df.stand.astype('category').cat.codes

In [ ]:
# fill na
df.condition = df.condition.fillna(0)

# drop na
df_model = df.dropna(subset=['launch_speed', 'launch_angle', 'altitude', 'barrel', 'stand', 'age_bat', 'temperature', 'condition', 'wind_int']).copy()

X = df_model[['launch_speed', 'launch_angle', 'altitude', 'barrel', 'stand', 'age_bat', 'temperature', 'condition', 'wind_int']]
y = df_model['home_run']

## Game info Dataset
add info from here

In [ ]:
import pandas as pd

In [ ]:
game = pd.read_csv('gameinfo.csv')
game

/tmp/ipython-input-1638284916.py:1: DtypeWarning: Columns (10,11,13,17,19,21,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  game = pd.read_csv('gameinfo.csv')


,gid,visteam,hometeam,site,date,number,starttime,daynight,innings,tiebreaker,...,vruns,hruns,wteam,lteam,line,batteries,lineups,box,pbp,season
0,LS3189904140,CHN,LS3,LOU03,18990414,0.0,0:00PM,day,NaN,NaN,...,15,1,CHN,LS3,y,both,y,y,NaN,1899
1,PHI189904140,WSN,PHI,PHI09,18990414,0.0,0:00PM,day,NaN,NaN,...,5,6,PHI,WSN,y,both,y,y,NaN,1899
2,BLN189904150,NY1,BLN,BAL07,18990415,0.0,0:00PM,day,NaN,NaN,...,3,5,BLN,NY1,y,both,y,y,NaN,1899
3,BRO189904150,BSN,BRO,NYC12,18990415,0.0,0:00PM,day,NaN,NaN,...,1,0,BSN,BRO,y,both,y,y,NaN,1899
4,CIN189904150,PIT,CIN,CIN05,18990415,0.0,0:00PM,day,NaN,NaN,...,5,2,PIT,CIN,y,both,y,y,NaN,1899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221168,LAN202410250,NYA,LAN,LOS03,20241025,0.0,5:08PM,night,9.0,NaN,...,3,6,LAN,NYA,y,both,y,y,y,2024
221169,LAN202410260,NYA,LAN,LOS03,20241026,0.0,5:08PM,night,9.0,NaN,...,2,4,LAN,NYA,y,both,y,y,y,2024
221170,NYA202410280,LAN,NYA,NYC21,20241028,0.0,8:08PM,night,9.0,NaN,...,4,2,LAN,NYA,y,both,y,y,y,2024
221171,NYA202410290,LAN,NYA,NYC21,20241029,0.0,8:08PM,night,9.0,NaN,...,4,11,NYA,LAN,y,both,y,y,y,2024
